**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# The Postgres Operator by Zalando

see https://github.com/zalando/postgres-operator

In [1]:
pwd

/minikube-host/training/notebooks


In [ ]:
git clone https://github.com/zalando/postgres-operator.git


In [2]:
cd postgres-operator
pwd

/minikube-host/training/notebooks/postgres-operator


In [ ]:
kubectl delete -f manifests/configmap.yaml  # configuration
kubectl delete -f manifests/operator-service-account-rbac.yaml  # identity and permissions
kubectl delete -f manifests/postgres-operator.yaml  # deployment
kubectl delete -f manifests/api-service.yaml  # operator API to be used by UI

In [3]:
helm install postgres-operator ./charts/postgres-operator

manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
NAME: postgres-operator
LAST DEPLOYED: Thu Jan 28 08:26:23 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


In [4]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


NAME                                 READY   STATUS    RESTARTS   AGE
postgres-operator-6f656b6b8b-bznhl   1/1     Running   0          15s


In [5]:
kubectl get pods -A

NAMESPACE     NAME                                       READY   STATUS    RESTARTS   AGE
default       postgres-operator-6f656b6b8b-bznhl         1/1     Running   0          20s
default       sleep-56bc8d768-qpvwt                      1/1     Running   0          16h
kube-system   calico-kube-controllers-55ffdb7658-6dnz6   1/1     Running   0          16h
kube-system   calico-node-k7vz4                          1/1     Running   0          16h
kube-system   coredns-74ff55c5b-lzz5q                    1/1     Running   0          16h
kube-system   etcd-minikube                              1/1     Running   0          16h
kube-system   kube-apiserver-minikube                    1/1     Running   0          16h
kube-system   kube-controller-manager-minikube           1/1     Running   0          16h
kube-system   kube-proxy-64jt7                           1/1     Running   0          16h
kube-system   kube-scheduler-minikube                    1/1     Running   0          16h
kube-syste

In [6]:
helm install postgres-operator-ui ./charts/postgres-operator-ui

NAME: postgres-operator-ui
LAST DEPLOYED: Thu Jan 28 08:26:54 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"


In [11]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"

NAME                                   READY   STATUS    RESTARTS   AGE
postgres-operator-ui-9d658b84c-9qnsl   1/1     Running   0          45s


In [12]:
kubectl port-forward svc/postgres-operator-ui 8081:80

Forwarding from 127.0.0.1:8081 -> 8081


In [ ]:
kubectl get all

In [ ]:
kubectl delete deployment nginx-deployment nginx-other-deployment nginx-strict nginx-hostnetwork-deployment

# In a separate terminal

`kubectl port-forward --address $(hostname -i) svc/postgres-operator-ui 8081:80`

In [14]:
kubectl get secret

NAME                                                      TYPE                                  DATA   AGE
default-token-kvx2n                                       kubernetes.io/service-account-token   3      17h
postgres-operator-token-m2fqq                             kubernetes.io/service-account-token   3      14m
postgres-operator-ui-token-hv7wn                          kubernetes.io/service-account-token   3      14m
postgres-pod-token-68vgp                                  kubernetes.io/service-account-token   3      97s
postgres.acid-mypg.credentials.postgresql.acid.zalan.do   Opaque                                2      97s
sh.helm.release.v1.postgres-operator-ui.v1                helm.sh/release.v1                    1      14m
sh.helm.release.v1.postgres-operator.v1                   helm.sh/release.v1                    1      15m
standby.acid-mypg.credentials.postgresql.acid.zalan.do    Opaque                                2      97s


In [26]:
kubectl get svc 

NAME                   TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)          AGE
acid-mypg              LoadBalancer   10.108.191.69   10.108.191.69   5432:31792/TCP   11m
acid-mypg-config       ClusterIP      None            <none>          <none>           10m
acid-mypg-repl         ClusterIP      10.111.59.22    <none>          5432/TCP         11m
kubernetes             ClusterIP      10.96.0.1       <none>          443/TCP          17h
postgres-operator      ClusterIP      10.108.16.199   <none>          8080/TCP         25m
postgres-operator-ui   ClusterIP      10.104.129.18   <none>          80/TCP           24m


In [25]:
kubectl get service acid-mypg-repl -o json

{
    "apiVersion": "v1",
    "kind": "Service",
    "metadata": {
        "creationTimestamp": "2021-01-28T08:39:46Z",
        "labels": {
            "application": "spilo",
            "cluster-name": "acid-mypg",
            "spilo-role": "replica",
            "team": "acid"
        },
        "managedFields": [
            {
                "apiVersion": "v1",
                "fieldsType": "FieldsV1",
                "fieldsV1": {
                    "f:metadata": {
                        "f:labels": {
                            ".": {},
                            "f:application": {},
                            "f:cluster-name": {},
                            "f:spilo-role": {},
                            "f:team": {}
                        }
                    },
                    "f:spec": {
                        "f:ports": {
                            ".": {},
                            "k:{\"port\":5432,\"protocol\":\"TCP\"}": {
                                "

In [28]:
export PGPASSWORD=$(kubectl get secret postgres.acid-mypg.credentials.postgresql.acid.zalan.do -o 'jsonpath={.data.password}' | base64 -d)
export PGSSLMODE=require

export PGHOST=$(kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}' )
echo $PGHOST

echo use
PGCOMMAND=$(echo PGPASSWORD=$PGPASSWORD PGSSLMODE=$PGSSLMODE  /usr/bin/psql  -h $PGHOST -U postgres)
echo $PGCOMMAND
echo to connect

10.111.59.22
use
PGPASSWORD=POO08lM2R3pph0M4bH2SzjgDkakiMuVexfju3ZrCzNJGRIlebcjIDXzJlq7U7jcN PGSSLMODE=require /usr/bin/psql -h 10.111.59.22 -U postgres
to connect


In [29]:
kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[*].ip}'

10.111.59.22

In [30]:
kubectl get all

NAME                                       READY   STATUS    RESTARTS   AGE
pod/acid-mypg-0                            1/1     Running   0          13m
pod/acid-mypg-1                            1/1     Running   0          12m
pod/acid-mypg-2                            1/1     Running   0          12m
pod/postgres-operator-6f656b6b8b-bznhl     1/1     Running   0          26m
pod/postgres-operator-ui-9d658b84c-9qnsl   1/1     Running   0          26m
pod/sleep-56bc8d768-qpvwt                  1/1     Running   0          17h

NAME                           TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)          AGE
service/acid-mypg              LoadBalancer   10.108.191.69   10.108.191.69   5432:31792/TCP   13m
service/acid-mypg-config       ClusterIP      None            <none>          <none>           12m
service/acid-mypg-repl         LoadBalancer   10.111.59.22    10.111.59.22    5432:32083/TCP   13m
service/kubernetes             ClusterIP      10.96.0.1       <none>   

In [31]:
kubectl exec -t  acid-mypg-0 -- /bin/bash -c "$PGCOMMAND -c 'select datname FROM pg_database'"

  datname  
-----------
 postgres
 template1
 template0
(3 rows)



In [ ]:
kubectl get pods


In [ ]:
kubectl logs acid-mypg-1 | grep  --color=always -E '$|leader'

In [ ]:
kubectl delete pod acid-mypg-0